In [136]:
import numpy as np
import pandas as pd
from itertools import chain, groupby
from collections import defaultdict
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer

## Подготовка данных (urls)

Я попытался учесть количество раз, с которым url встречается в браузерной истории пользователя.

In [2]:
urls_train_df = pd.read_csv('kaggle_data/url_domain_train', header=None, delimiter='\t')
urls_train_df.columns = ['id', 'url', 'count']
urls_train_df.head()

,id,url,count
0,000000014B60815F65B38258011B6C01,login.rutracker.org,1
1,000000014B60815F65B38258011B6C01,rutracker.org,4
2,000000014C03DA2A47AC433A0C755201,admin.tour-spb.net,1
3,000000014C03DA2A47AC433A0C755201,czinfo.ru,1
4,000000014C03DA2A47AC433A0C755201,forumsostav.ru,1


In [3]:
urls_train_df = pd.DataFrame(urls_train_df.groupby('id').apply(lambda df: list(zip(df['url'], df['count']))))
urls_train_df['id'] = urls_train_df.index
urls_train_df.index = range(len(urls_train_df))
urls_train_df.columns = ['urls', 'id']
urls_train_df.head()

,urls,id
0,"[(id.rambler.ru, 1), (mail.rambler.ru, 1), (r0...",000000013CB5719C0000A2C90002C101
1,"[(1prime.ru, 1), (autorambler.ru, 1), (chellak...",00000001442BE24000001B7D00F50801
2,"[(bosch-korolev.ru, 1)]",00000001448580F800003F1B31FB0901
3,"[(aptekanizkihcen.ua, 1), (colady.ru, 1), (gor...",0000000145BDB2FF000157971645E901
4,"[(astrorok.ru, 1), (diets.ru, 1), (edaplus.inf...",000000014602771F0000DB9359714C01


In [4]:
age_train_df = pd.read_csv('kaggle_data/age_profile_train', header=None, delimiter='\t')
age_train_df.columns = ['id', 'age']
print("Max age: {}".format(max(age_train_df['age'])))
print("Min age: {}".format(min(age_train_df['age'])))
age_train_df.head()

Max age: 99
Min age: 0


,id,age
0,000000013CB5719C0000A2C90002C101,53
1,00000001442BE24000001B7D00F50801,48
2,00000001448580F800003F1B31FB0901,28
3,0000000145BDB2FF000157971645E901,44
4,000000014602771F0000DB9359714C01,48


In [50]:
train_df = urls_train_df.merge(age_train_df, on='id', how='left')
print(len(train_df))
train_df[train_df['age'] == 58]

118603


,urls,id,age
33,"[(45fss.ru, 2), (fss.ru, 2), (kurgan.spravker....",000000014993E4FE0AF232A20A374C01,58
117,"[(mail.rambler.ru, 1)]",000000014BB63F59B332AD2502C15401,58
197,"[(freehouse.ru, 2), (topnomer.ru, 1)]",000000014D00FADA5629475402FAE401,58
290,"[(mail.rambler.ru, 1)]",000000014E12FCD1517F2108000CF301,58
328,"[(auto.vesti.ru, 1), (filmpro.ru, 1), (inforea...",000000014E22A4870B372EA500FA2E01,58
546,"[(1prime.ru, 1), (assessor.ru, 1), (base.consu...",000000014FA4F5FE7B2C063702E1B901,58
560,"[(assets.adobedtm.com, 1), (championat.com, 3)...",000000014FB2430800FF6001012F0501,58
710,"[(ecson.ru, 1), (mail.rambler.ru, 19), (o-voyn...",00000001504EF1CB4215469206A3F901,58
733,"[(1tv.ru, 2), (belaruspartisan.org, 1), (dieta...",0000000150614BA696E0528101171A01,58
832,"[(bom-bom.ru, 1), (ffclub.ru, 1), (horoscopes....",0000000150BC9E9414058556001BE901,58


## Снижение размерности

Здесь я попытался реализовать то, что было описано на семинаре.
Для каждого $i \in \{0, \ldots, 9\}$ я считаю $P(age \in [10 i, 10 (i + 1)) | url)$. В результате у меня получается 10 словарей с условными вероятностями для каждого url и для каждого отрезка возраста.

In [6]:
len(train_df)

118603

In [7]:
def calc_probs(train_df, from_age, to_age):
    total_visits = defaultdict(float)
    cond_visits = defaultdict(float)
    for domains, age in zip(train_df.urls.values, train_df.age.values):
        for domain, count in domains:
            if age in range(from_age, to_age):
                cond_visits[domain] += count
            total_visits[domain] += count
    for key in cond_visits:
        cond_visits[key] /= total_visits[key]
    return cond_visits

conditionals = []
for i in range(10):
    print(i)
    conditionals.append(calc_probs(train_df, 10 * i, 10 * (i + 1)))

0
1
2
3
4
5
6
7
8
9


Далее, вектор фичей, которые отвечают за пользователя, строится так: для каждого отрезка возраста строится гистограмма из 100 бакетов, и все они сливаются. Получается вектор длины 1000. К сожалению некоторые компоненты этих векторов получаются зависимыми, но я так и не смог придумать, как учитывать одновременно много промежутков возростов и сделать так, чтобы фичи были независимы. Можно, конечно, сделать $P(age \in [0, 25] | url)$, но так получается, что мы слишком сильно теряем информации из входных данных.

In [115]:
def transform(urls):
    x = np.zeros((1000,))
    for url, count in urls:
        for i in range(10):
            if url in conditionals[i]:
                probability = conditionals[i][url]
                x[100 * i + int((probability - 1e-6) * 100)] += count
            
    return x / np.linalg.norm(x)

In [20]:
y = train_df.age.values
X = []
for val in train_df.urls.values:
    X.append(transform(val))

In [21]:
len(y)

118603

In [22]:
X = np.array(X)
X.shape

(118603, 1000)

In [69]:
X[1]

array([ 0.01667593,  0.38910512,  0.04446916,  0.        ,  0.        ,
        0.        ,  0.        ,  0.00555864,  0.        ,  0.        ,
        0.00555864,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

## Обучение модели

Здесь я написал небольшую cross_validation, я попробовал Ridge (линейная модель с L2-регуляризацией), Lasso (L1 резуляризация, в теории должна лучше работать).

In [107]:
regressions = [Lasso, Ridge]
for alpha in [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]:
    for reg in regressions:
        r = reg(alpha=alpha)
        print(reg, alpha)
        scores = (-cross_val_score(r, X, y, scoring='mean_squared_error'))
        print(scores)
        print(sum(scores) / 3)
        print('----------')

/Users/astepanov/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/Users/astepanov/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


<class 'sklearn.linear_model.coordinate_descent.Lasso'> 0.0001
[ 121.83068989  130.3887478    96.73304434]
116.317494011
----------
<class 'sklearn.linear_model.ridge.Ridge'> 0.0001
[ 122.53057224  130.99959883  101.18705273]
118.239074601
----------
<class 'sklearn.linear_model.coordinate_descent.Lasso'> 0.001
[ 125.70604072  132.07278338  101.14903567]
119.642619923
----------
<class 'sklearn.linear_model.ridge.Ridge'> 0.001
[ 122.10582529  130.77051804   99.1455297 ]
117.340624344
----------
<class 'sklearn.linear_model.coordinate_descent.Lasso'> 0.01
[ 136.52766534  138.04645292  112.26152955]
128.945215939
----------
<class 'sklearn.linear_model.ridge.Ridge'> 0.01
[ 121.4773583   130.57298842   97.62278189]
116.557709536
----------
<class 'sklearn.linear_model.coordinate_descent.Lasso'> 0.1
[ 155.44611844  150.9030644   133.95268106]
146.767287965
----------
<class 'sklearn.linear_model.ridge.Ridge'> 0.1
[ 121.7787172   130.47703769   97.42165158]
116.559135488
----------
<class '

/Users/astepanov/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


Для сравнения ElasticNet (композиция L2 и L1 резуляризаций)

In [113]:
r = ElasticNet(alpha=0.01, l1_ratio=0.01)
scores = (-cross_val_score(r, X, y, scoring='mean_squared_error'))
print(scores)
print(sum(scores) / 3)

[ 148.16604999  145.71214886  127.74198659]
140.540061811


## Отправка Решения

Было решено использовать Ridge(alpha=0.1). К сожалению, не смотря на хороший скор на cross_validation, это решение набрало на kaggle 148.

In [114]:
reg = Ridge(alpha=0.1)
reg.fit(X, y)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [123]:
urls_test_df = pd.read_csv('kaggle_data/url_domain_test', header=None, delimiter='\t')
urls_test_df.columns = ['id', 'url', 'count']
urls_test_df.head()

,id,url,count
0,0000000151004FF4ADD746DA10685A01,afisha.ru,2
1,0000000151004FF4ADD746DA10685A01,aif.ru,1
2,0000000151004FF4ADD746DA10685A01,aimfar.solution.weborama.fr,1
3,0000000151004FF4ADD746DA10685A01,alkotest.ru,1
4,0000000151004FF4ADD746DA10685A01,aptekamos.ru,1


In [124]:
urls_test_df = pd.DataFrame(urls_test_df.groupby('id').apply(lambda df: list(zip(df['url'], df['count']))))
urls_test_df['id'] = urls_test_df.index
urls_test_df.index = range(len(urls_test_df))
urls_test_df.columns = ['urls', 'id']
urls_test_df.head()

,urls,id
0,"[(1000bankov.ru, 1), (1tv.ru, 8), (4put.ru, 1)...",000000014A02348E701552980349FF01
1,"[(autorambler.ru, 1), (bilettorg.ru, 1), (dsol...",000000014A10EA183BF8594A0B2AB201
2,"[(photosight.ru, 6), (rambler.ru, 6)]",000000014A4FE5C33A929D4C26943601
3,"[(base.consultant.ru, 1), (dogovor-obrazets.ru...",000000014B7BB9957784A9BC0AC9F401
4,"[(assessor.ru, 2), (audit-it.ru, 1), (base.gar...",000000014C7749F896D82C2B01E8B801


In [125]:
X_test = []
for val in urls_test_df.urls.values:
    X_test.append(transform(val))
X_test = np.array(X_test)

In [126]:
X_test.shape

(19974, 1000)

In [127]:
y_pred = reg.predict(X_test)

In [128]:
y_pred

array([ 39.06696015,  42.0904562 ,  51.14819456, ...,  37.54280448,
        37.19906284,  38.12296796])

In [129]:
urls_test_df['age'] = y_pred

In [130]:
urls_test_df = urls_test_df[['id', 'age']]
urls_test_df.columns = ['Id', 'age']

In [131]:
urls_test_df.head()

,Id,age
0,000000014A02348E701552980349FF01,39.066960
1,000000014A10EA183BF8594A0B2AB201,42.090456
2,000000014A4FE5C33A929D4C26943601,51.148195
3,000000014B7BB9957784A9BC0AC9F401,40.113299
4,000000014C7749F896D82C2B01E8B801,37.109740


In [132]:
random_sol = pd.read_csv('kaggle_data/random_solution.csv')
miss_idx = set(random_sol.Id.values) - set(urls_test_df.Id.values)
miss_df = pd.DataFrame(list(zip(list(miss_idx), np.ones(len(miss_idx)))))
miss_df.columns = ['Id', 'age']

In [133]:
urls_test_df = urls_test_df.append(miss_df, ignore_index=True)

In [134]:
urls_test_df.to_csv('kaggle_data/solution.csv', index=False)

In [135]:
!wc -l kaggle_data/solution.csv

   19980 kaggle_data/solution.csv


FIN.